# RQ1

In [318]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 200)

dic_approach = {'lgb': 'LGMP', 
               'random_apfd': 'Random', 
               'deepGini_apfd': 'DeepGini', 
               'leastConfidence_apfd': 'LC', 
               'margin_apfd': 'Margin', 
               'rf': 'RFMP', 
               'xgb': 'XGMP', 
               'lr': 'LRMP'}
dic_other = {'lr': 'LR', 
             'xgboost': 'XGBoost', 
             'lgb': 'LightGBM', 
             'rf': 'RF', 
             'bank': 'Bank', 
             'heart': 'Heart', 
             'stroke': 'Stroke', 
             'adult': 'Adult'
            }

def get_path(path_dir_compile):
    path_list = []
    if os.path.isdir(path_dir_compile):
        for root, dirs, files in os.walk(path_dir_compile, topdown=True):
            for file in files:
                file_absolute_path = os.path.join(root, file)
                if file_absolute_path.endswith('.csv'):
                    path_list.append(file_absolute_path)
    return path_list

path_list = get_path('result/original_res/')
compare_list = [i for i in path_list if 'compare' in i]
compare_model = [i.split('/')[-1].split('_')[0] for i in compare_list]
compare_data = [i.split('/')[-1].split('_')[1] for i in compare_list]

model_list = [i for i in path_list if 'compare' not in i]
model_name = [i.split('/')[-1].split('_')[0] for i in model_list]
model_data = [i.split('/')[-1].split('_')[1] for i in model_list]

data_list = []
for i in range(len(compare_list)):
    tmp_df = pd.read_csv(compare_list[i])
    tmp_df['model'] = compare_model[i]
    tmp_df['data'] = compare_data[i]
    data_list.append(tmp_df)
df_compare = pd.concat(data_list, ignore_index=True)

data_list = []
for i in range(len(model_list)):
    tmp_df = pd.read_csv(model_list[i])
    tmp_df['model'] = model_name[i]
    tmp_df['data'] = model_data[i]
    del tmp_df['mutation_feature_apfd']
    del tmp_df['mutation_model_apfd']
    del tmp_df['fusion_2_feature_apfd']
    tmp_df = tmp_df.rename(columns={"fusion_3_feature_apfd": "apfd"})
    data_list.append(tmp_df)
df_model = pd.concat(data_list, ignore_index=True)

df = pd.concat([df_compare, df_model], ignore_index=True)
df['Approach'] = [dic_approach[i] for i in df['Approach']]
df['model'] = [dic_other[i] for i in df['model']]
df['data'] = [dic_other[i] for i in df['data']]
df['apfd'] = [str(i)[:6] for i in df['apfd']]

approach_list = ['Random', 'Margin', 'LC', 'DeepGini', 'LRMP',  'RFMP', 'XGMP', 'LGMP',]
data_list = ['Adult', 'Bank', 'Heart', 'Stroke']
model_list = ['LR', 'RF', 'XGBoost', 'LightGBM']

res_list = []
for approach in approach_list:
    tmp_res = [approach]
    for data in data_list:
        for model in model_list:
            apfd = df[(df['Approach']==approach)&(df['data']==data)&(df['model']==model)]['apfd'].values[0]
            tmp_res.append(apfd)
    res_list.append(tmp_res)
df_re = pd.DataFrame(data=res_list, columns=['Approach']+model_list*4)
df_re.to_excel('result/table//RQ1-1.xlsx', index=False)
df_re

,Approach,LR,RF,XGBoost,LightGBM,LR,RF,XGBoost,LightGBM,LR,RF,XGBoost,LightGBM,LR,RF,XGBoost,LightGBM
0,Random,0.4975,0.4924,0.4964,0.4992,0.4995,0.5051,0.4881,0.4994,0.4971,0.4993,0.5061,0.4977,0.4978,0.5074,0.4865,0.4959
1,Margin,0.7366,0.6639,0.7873,0.8048,0.7987,0.7367,0.8262,0.8471,0.7667,0.8259,0.9082,0.8494,0.5959,0.8573,0.7579,0.6942
2,LC,0.7366,0.6639,0.7873,0.8048,0.7987,0.7367,0.8262,0.8471,0.7639,0.8259,0.9069,0.8561,0.5959,0.8573,0.7579,0.6942
3,DeepGini,0.7366,0.6699,0.7875,0.8047,0.7987,0.7320,0.8255,0.8467,0.7608,0.8197,0.9066,0.8566,0.5959,0.8661,0.7577,0.6948
4,LRMP,0.7342,0.7749,0.7451,0.7754,0.8127,0.7131,0.8059,0.8150,0.8030,0.8240,0.7558,0.7627,0.6132,0.9774,0.7171,0.6830
5,RFMP,0.8053,0.7377,0.7947,0.8090,0.8768,0.7448,0.8755,0.8810,0.9264,0.8664,0.9507,0.9613,0.8437,0.9842,0.8997,0.9078
6,XGMP,0.8244,0.7775,0.8161,0.8232,0.8785,0.7526,0.8667,0.8821,0.9228,0.8972,0.9303,0.9474,0.8443,0.9780,0.8992,0.9078
7,LGMP,0.8258,0.7943,0.8188,0.8245,0.8783,0.7215,0.8682,0.8828,0.9192,0.8669,0.9328,0.9450,0.8412,0.9771,0.8975,0.9070


In [319]:
res_list = []
res_np = df_re.to_numpy()[:, 1:].astype(np.float64)
improvement = (res_np[4]-res_np[3])/res_np[3]
improvement = ['LRMP']+[str(i*100)[:5] for i in improvement]
res_list.append(improvement)
improvement =(res_np[5]-res_np[3])/res_np[3]
improvement = ['RFMP']+[str(i*100)[:5] for i in improvement]
res_list.append(improvement)
improvement =(res_np[6]-res_np[3])/res_np[3]
improvement = ['XGMP']+[str(i*100)[:5] for i in improvement]
res_list.append(improvement)
improvement =(res_np[7]-res_np[3])/res_np[3]
improvement =['LGMP']+[str(i*100)[:5] for i in improvement]
res_list.append(improvement)
df_re = pd.DataFrame(data=res_list, columns=['Approach']+model_list*4)
df_re.to_excel('result/table/RQ1-2.xlsx', index=False)
df_re

,Approach,LR,RF,XGBoost,LightGBM,LR,RF,XGBoost,LightGBM,LR,RF,XGBoost,LightGBM,LR,RF,XGBoost,LightGBM
0,LRMP,-0.32,15.67,-5.38,-3.64,1.752,-2.58,-2.37,-3.74,5.546,0.524,-16.6,-10.9,2.903,12.85,-5.35,-1.69
1,RFMP,9.326,10.12,0.914,0.534,9.778,1.748,6.056,4.051,21.76,5.697,4.864,12.22,41.58,13.63,18.74,30.65
2,XGMP,11.91,16.06,3.631,2.298,9.991,2.814,4.990,4.180,21.29,9.454,2.614,10.60,41.68,12.91,18.67,30.65
3,LGMP,12.10,18.56,3.974,2.460,9.966,-1.43,5.172,4.263,20.82,5.758,2.889,10.31,41.16,12.81,18.45,30.54


# RQ2

In [11]:
import pandas as pd
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', 200)

dic_approach = {'lgb': 'LGMP', 
               'random_apfd': 'Random', 
               'deepGini_apfd': 'DeepGini', 
               'leastConfidence_apfd': 'LC', 
               'margin_apfd': 'Margin', 
               'rf': 'RFMP', 
               'xgb': 'XGMP', 
               'lr': 'LRMP'}
dic_other = {'lr': 'LR', 
             'xgboost': 'XGBoost', 
             'lgb': 'LightGBM', 
             'rf': 'RF', 
             'bank': 'Bank', 
             'heart': 'Heart', 
             'stroke': 'Stroke', 
             'adult': 'Adult'
            }

def get_path(path_dir_compile):
    path_list = []
    if os.path.isdir(path_dir_compile):
        for root, dirs, files in os.walk(path_dir_compile, topdown=True):
            for file in files:
                file_absolute_path = os.path.join(root, file)
                if file_absolute_path.endswith('.csv'):
                    path_list.append(file_absolute_path)
    return path_list

path_list = get_path('result/fairness_res/')
compare_list = [i for i in path_list if 'compare']
compare_model = [i.split('/')[-1].split('_')[0] for i in compare_list]
compare_data = [i.split('/')[-1].split('_')[1] for i in compare_list]

model_list = [i for i in path_list if 'compare' not in i]
model_name = [i.split('/')[-1].split('_')[0] for i in model_list]
model_data = [i.split('/')[-1].split('_')[1] for i in model_list]

data_list = []
for i in range(len(compare_list)):
    tmp_df = pd.read_csv(compare_list[i])
    tmp_df['model'] = compare_model[i]
    tmp_df['data'] = compare_data[i]
    data_list.append(tmp_df)
df_compare = pd.concat(data_list, ignore_index=True)
tmp_df.head()


# data_list = []
# for i in range(len(model_list)):
#     tmp_df = pd.read_csv(model_list[i])
#     tmp_df['model'] = model_name[i]
#     tmp_df['data'] = model_data[i]
#     del tmp_df['mutation_feature_apfd']
#     del tmp_df['mutation_model_apfd']
#     del tmp_df['fusion_2_feature_apfd']
#     tmp_df = tmp_df.rename(columns={"fusion_3_feature_apfd": "apfd"})
#     data_list.append(tmp_df)
# df_model = pd.concat(data_list, ignore_index=True)

# df = pd.concat([df_compare, df_model], ignore_index=True)
# df['Approach'] = [dic_approach[i] for i in df['Approach']]
# df['model'] = [dic_other[i] for i in df['model']]
# df['data'] = [dic_other[i] for i in df['data']]
# df['apfd'] = [str(i)[:6] for i in df['apfd']]

# approach_list = ['Random', 'Margin', 'LC', 'DeepGini', 'LRMP',  'RFMP', 'XGMP', 'LGMP',]
# data_list = ['Adult', 'Bank', 'Heart', 'Stroke']
# model_list = ['LR', 'RF', 'XGBoost', 'LightGBM']

# res_list = []
# for approach in approach_list:
#     tmp_res = [approach]
#     for data in data_list:
#         for model in model_list:
#             apfd = df[(df['Approach']==approach)&(df['data']==data)&(df['model']==model)]['apfd'].values[0]
#             tmp_res.append(apfd)
#     res_list.append(tmp_res)
# df_re = pd.DataFrame(data=res_list, columns=['Approach']+model_list*4)

# df_re.head()

,Approach,mutation_feature_apfd,mutation_model_apfd,fusion_2_feature_apfd,fusion_3_feature_apfd,model,data
0,lr,0.591662,0.610236,0.670293,0.740498,lr,fairness
1,rf,0.589610,0.612024,0.671932,0.807367,lr,fairness
2,xgb,0.589617,0.612550,0.672006,0.827015,lr,fairness
3,lgb,0.588963,0.612321,0.673015,0.829485,lr,fairness
